# This is a consolidated version of all the data


In [6]:
import base64
import datetime
import pandas as pd
import json
from google.cloud import bigquery

client = bigquery.Client(location="US")
print("Client creating using default project: {}".format(client.project))

Client creating using default project: research-311404


**AI2P data is suuposed to be 'good'**


 So check a three-year inventory on the D data
 This time, put the adjusted D data in there as well

In [7]:
#this dataset: research-311404.AI2Patent_TreeConstruction_202308.AI2P_w_paper_full_inversed
# Used the full AI2P Tree and appended paper data
sql = """
        CREATE TABLE research-311404.AI2Patent_Tree_Consolidated.AI2P_patent_level_min_full
        AS SELECT patentid,year,  inverse_D, (SUM(D_Cf)/NULLIF(SUM(C_f),0)) AS D_Cf, (SUM(D_Hit)/NULLIF(SUM(Hit_10pct),0)) AS D_Hit, (SUM(D_Disruption)/NULLIF(SUM(Disruption),0)) AS D_Disruption, (SUM(D_Exposure)/NULLIF(SUM(Exposure),0)) AS D_Exposure, (SUM(D_time)/NULLIF(SUM(Timelag_inv),0)) AS D_time FROM  `research-311404.AI2Patent_TreeConstruction_202308.AI2P_w_paper_full_inversed`

        group by patentid,year,inverse_D
        """

job = client.query(sql)  # API request.
job.result()  # Waits for the query to finish.

In [9]:
# AI2P dataset contains from year 1980-2021. Therefore to have available data from t-3 to t+3, 
# we need to construct year-level data (t) ranging from 1983 to 2017
# Construting AI2P SCORE
# D values are calculated on the average of the pre-three year patent inventory
for i in range(35):
    start_year = i + 1980
    end_year = i + 1983
    sql = f"""
        CREATE TABLE research-311404.temp.AI2P_{end_year}
        AS SELECT inventor_id, {end_year} AS fyear,  AVG(inverse_D) AS inverse_D, AVG(D_Cf) AS D_Cf, AVG(D_Hit) AS D_Hit, AVG(D_Disruption) AS D_Disruption, AVG(D_Exposure) AS D_Exposure ,AVG(D_time)  AS  D_time FROM
        ((SELECT patentid,year,  inverse_D, D_Cf, D_Hit, D_Disruption, D_Exposure,D_time FROM  `research-311404.AI2Patent_Tree_Consolidated.AI2P_patent_level_min_full`)t1
        LEFT JOIN
        (SELECT patent_id,inventor_id FROM `patents-public-data.patentsview.patent_inventor` )t2
        ON t1.patentid = t2.patent_id
        )WHERE year >= {start_year} and year <= {end_year}
        group by inventor_id
        """

    job = client.query(sql)  # API request.
    job.result()  # Waits for the query to finish.


In [11]:
# Consolidate to a single file
sql = """
        CREATE TABLE research-311404.AI2Patent_Tree_Consolidated.AI2P
        AS SELECT * FROM `research-311404.temp.AI2P_*`
        """

job = client.query(sql)  # API request.
job.result()

In [ ]:
# The respective examiner toughness metric should also be constructed here
# We use this table: research-311404.Examiner_toughness_by_date.examiner_table_fillna
# Limited by OCE PAIR dataset: 9,817,693 application_number-examiner pairs
# 9,739,630 not Null examiner names
# 
# This table contains count information for all 

In [ ]:
# Construting Dependent variables: They should be three years looking forward: 
# D values are calculated on the average of the pre-three year patent inventory